# Problem definition

We wish to minimize

$$ I(u,v) = \frac{\theta}{2} \int_{\omega} Q_2(\nabla_s u + \tfrac{1}{2} \nabla v \otimes \nabla v) \mathrm{d}x
   + \frac{1}{24} \int_{\omega} Q_2(\nabla^2 v - B) \mathrm{d}x, $$

with $B \in \mathbb{R}^{2 \times 2}$, e.g. the identity matrix, and $Q_2$ a quadratic form, e.g. (isotropic material):

$$ Q_2 (F) = 2 \mu | \operatorname{sym} F |^2 + \frac{2 \mu \lambda}{2 \mu + \lambda}
   \operatorname{tr}^2 F, \quad F \in \mathbb{R}^{2 \times 2}, $$

or, for a specific choice of constants, the simpler $Q_2(F) = |F|^2$.
  
We work in $P_1$ with the constraints of zero mean and zero mean antisymmetric gradient. Because we only have $C^0$ elements we set $z$ for $\nabla v$ and minimize instead

$$ J(u,z) = \frac{\theta}{2} \int_{\omega} Q_2(\nabla_s u + \tfrac{1}{2} z \otimes z) \mathrm{d}x 
          + \frac{1}{24} \int_{\omega} Q_2\nabla z - B) \mathrm{d}x 
          + \mu \int_{\omega} |\mathrm{curl}\ z|^{2} \mathrm{d}x, $$

then recover the vertical displacements (up to a constant) by minimizing

$$ F(p,q) = \tfrac{1}{2} || \nabla p - q ||^2 + \tfrac{1}{2} || q - z ||^2. $$

This we do by solving the linear problem $D F = 0$.

Minimization of the energy functional $J$ is done via gradient descent and a line search. In particular, at each timestep we compute $d_t w \in W $ such that for all $\tau \in W$:

$$ (d_t w, \tau)_{H^1_0 \times H^2_0} = -DJ(w_t)[\tau] $$

Note that it is essential to use the full scalar product (or the one corresponding to the seminorms? check this) or we run into issues at the boundaries (to see this start with zero displacements and integrate by parts).(Also: the proper Riesz representative will only be obtained with correct scalar product).

A decoupled gradient descent in each component does not work, probably because the functional is not separately convex (see Bartels' book, p. 110, remark (iv)).

In plane displacements and gradients of out of plane displacements form a mixed function space $U \times Z$. We also have another scalar space $V$ where the potential of the out of plane gradients lives. The model is defined and solved in `run_model()`.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as pl
from incense import ExperimentLoader

# Exploring the range of $\theta$

In [ ]:
loader = ExperimentLoader(mongo_uri='mongo:27017', db_name='lvk')

experiments = loader.find_by_config_key("init", "ani_parab")

In [ ]:
def average_last(ss):
    return ss[-10:].mean()

res = experiments.project(on=["config.theta",
                              "config.mu_scale",
                              {"metrics.J": average_last,
                               "metrics.symmetry": average_last,
                               "metrics.constraint": average_last,
                               "metrics.Kxx": average_last,
                               "metrics.Kxy": average_last,
                               "metrics.Kyy": average_last,
                               "metrics.alpha": len}])

# HACK: there is probably a better way of counting (and I could use df.rename())
res['steps'] = res['alpha_len']
del res['alpha_len']

n = len(res)

In [ ]:
pl.figure(figsize=(16,8))
pl.plot(res['theta'][:n], res['symmetry_average_last'][:n], marker='.', label='symmetry')
pl.xlabel('$\\theta$', fontsize=16)
pl.ylabel('symmetry', fontsize=16)
pl.legend()
pl.tick_params(axis='both', which='major', labelsize=16)

In [ ]:
pl.figure(figsize=(16,8))
pl.plot(res['theta'][:n], res['Kxx_average_last'][:n], marker='.', label="$K_{xx}$")
pl.plot(res['theta'][:n], res['Kxy_average_last'][:n], marker='.', label="$K_{xy}$")
pl.plot(res['theta'][:n], res['Kyy_average_last'][:n], marker='.', label="$K_{yy}$")
pl.xlabel('$\\theta$', fontsize=16)
pl.ylabel('curvatures', fontsize=16)
pl.legend()
pl.tick_params(axis='both', which='major', labelsize=16)
#pl.savefig('theta-symmetry.eps')

With increasing $\theta$ we expect the symmetry of the solution to be ever more violated until it is cylindrical rather than parabolic. We see a sharp increase around $\theta = 78$. Note that this could mean little since
* We use a poor criterion for symmetry (we are just taking the quotient of the principal axes)
* Used solutions are not necessarily minima (gradient descent might not converge to $\epsilon_{\text{stop}}$ precision)
* ...

In [ ]:
pl.figure(figsize=(16,8))
pl.plot(res['theta'][:n], res['constraint_average_last'][:n]*1e4, marker='.', label='$|curl|$')
#pl.plot(res['theta'][:n], res['mu_scale'][:n], marker='x', label='$\mu_\epsilon scale$')
pl.xlabel('$\\theta$', fontsize=16)
pl.ylabel('$|curl|*10^4$', fontsize=16)
pl.tick_params(axis='both', which='major', labelsize=16)
_ = pl.legend()
#pl.savefig('theta-curl.eps')

In [ ]:
pl.figure(figsize=(16,8))
pl.plot(res['theta'][:n], res['steps'][:n], marker='.')
pl.xlabel('$\\theta$', fontsize=16)
pl.ylabel('Num. steps', fontsize=16)
pl.tick_params(axis='both', which='major', labelsize=16)